# Imports

In [1]:
import os

from multi_agent_llm import AGOT, OpenAILLM

from pydantic import BaseModel, Field

# Set OpenAI API Key

In [2]:
os.environ['OPENAI_API_KEY'] = open('../openai_token.key').read().strip()

# Initialize an LLM client

In [3]:
llm = OpenAILLM("gpt-4o-mini", temperature=0.3)

# Define an output schema

In [4]:
class QueryAnswer(BaseModel):
    explanation: str = Field(description="Explanation of the answer.")
    answer: str = Field(description="Final multiple-choice answer.")
    answer_label: str = Field(description="Label of the answer. Either A, B, C, or D.")

# Sample query using Adaptive Graph of Thoughts (AGoT)

In [5]:
agot = AGOT(
    llm=llm,
    max_depth=1,
    max_num_layers=3,
    max_new_tasks=3,
    max_concurrent_tasks=10,
)

In [6]:
question = """
Observations of a quasar across the electromagnetic spectrum are being carried out.
Focusing on the near infrared and optical spectrum, there is a peak at a wavelength of about 790 nm,
and at shorter wavelengths < 790 nm the flux drops significantly.

If we lived in a universe defined by a Lambda-CDM model such that the current Hubble constant is 70 km / s / Mpc,
the matter density parameter is 0.3, the dark energy density parameter is 0.7, and the universe is flat,
what can be assumed about the value of the comoving distance (for scale factor a=1) of this quasar from
the Earth?

A. 6 Gpc
B. 7 Gpc
C. 8 Gpc
D. 9 Gpc
"""

response = await agot.run_async(question, schema=QueryAnswer)

In [9]:
answer = response.final_answer

print(f"ANSWER: {answer.answer_label} - {answer.answer}\n")
print(f"EXPLANATION: {answer.explanation}")

ANSWER: C - 8 Gpc

EXPLANATION: The analysis of the quasar's distance was based on comoving distance calculations employing the Lambda-CDM model parameters. Given the quasar's peak emission wavelength at 790 nm and estimated redshift effects from existing literature, the comoving distance is derived using integration over the redshift, leading to a calculated distance consistent with values observed in other quasars. Previous studies have established that quasars with similar characteristics typically lie at distances ranging from 6 to 9 Gpc. The comprehensive review of analyses culminated in a well-supported final distance assessment placing the quasar at approximately 8 Gpc.
